# Basic LangChain concepts
In this notebook we will explore LangChain framework capabilities and use it to interact with a LLM. You can either install [Ollama](https://ollama.com/) on your computer, use one of the models provided by [HuggingFace](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines) or create an [OpenAI API token](https://openai.com/). Remember to install the requirements from the `requirements.txt` file using `pip install -r requirements.txt`.

## Basic LLM interactions

### Model loading
Loading a model and using it with LangChain is extremely easy. All models implement [Runnable interface](https://python.langchain.com/docs/integrations/llms/) which makes it easy to replace them in the chain. Define your selected model and proceed to the exercises.

In [ ]:
# Use Ollama model. Requires to install the Ollama package: https://ollama.com/
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")

In [ ]:
# Use HuggingFace model. Requires to install extra dependencies with `pip install transformers sentence_transformers torch`
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="gpt2",
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 10},
# )

In [ ]:
# Use OpenAI model. Requires API keys and extra dependencies `pip install langchain-openai`
# %export OPENAI_API_KEY="..."
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()

To query the model you can simply run the `invoke` method. Experiment with different prompts to get an intuition about the model behavior. 

In [ ]:
llm.invoke("Hey, nice to meet you")

### Prompt templates
LangChain allows to define a prompt template that will be reused to query a model. It is a very convenient mechanism that allows you to deliver instructions and extra context to the model. You can experiment with system messages to influence the behavior of the model and assign it different roles. Try a couple of system prompts to see what you can do with it.

A great feature of LangChain is its [expression language](https://python.langchain.com/docs/expression_language/get_started). It allows defining chains of operations that will be performed sequentially. Here we can connect a prompt template with an LLM such that we don't have to provide repeatable content of the query over and over again.

#### Simple prompts

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
Answer the following question: {input}
"""

prompt = PromptTemplate.from_template(template)
prompt.invoke("How are you doing?")

**Exercise:** create a prompt template that includes `context` and `question` fields and run it using the `invoke` method. To achieve that you will have to run the method with a dictionary as an input. 

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
### TODO: create prompt template with `context` and `question` fields ###
"""

prompt = PromptTemplate.from_template(template)
prompt.invoke(### TODO: invoke template with proper parameters ###)

#### Chaining prompt and model
To build a chain of operations you can simply use `|` pipe operator. Now you can chain your prompt with the llm to get an answer to your question given the context!

In [ ]:
chain = prompt | llm
chain

In [ ]:
reply = chain.invoke(
    ### TODO: Invoke the chain with proper parameters ###
)
print(reply)

#### Chat prompts
Chat prompts may include extra instructions like system messages about a role the agent is playing. This message will affect the model behavior. It is a place where you can put additional security guards ie. forbidden actions. You can also influence the model to behave in a particular way.

**Exercise:** Experiment with the system message and see how the model replies change.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class agent and you serve your customers well"),
    ("user", "Here is my question: {input}")
])

chain = prompt | llm 
chain

In [ ]:
print(chain.invoke("I want to cancel my order"))

## Embeddings
For semantic search we need to create embeddings for our documents. In this section we will experiment with embeddings of small portions of text to get a better understanding of how similarity works. LangChain provides a great interface for [embedding models](https://python.langchain.com/docs/modules/data_connection/text_embedding/) and integrates smoothly with Ollama, OpenAI or HuggingFace sentence transformers. 

**Exercise:** Create embeddings of your sentences and check what is the similarity score.

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(show_progress=True)

In [ ]:
# OpenAI Embeddings, requires OpenAI API key. 
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings()

In [ ]:
# from langchain_community.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
import numpy as np

def cosine_similarity(vector1: list, vector2: list) -> float:
    # Convert lists to NumPy arrays
    v1 = np.array(vector1)
    v2 = np.array(vector2)
    
    # Compute the dot product between the two vectors
    dot_product = np.dot(v1, v2)
    
    # Compute the L2 norms (Euclidean norms) of each vector
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    # Calculate the cosine similarity
    return np.dot(v1,v2)/(norm_v1*norm_v2)

    
# For short sentences it is easy to introduce some noise. 
# Ie. model assigns big similarity to sentences that ends with a dot.
query1 = embeddings.embed_query("My package was not delivered")
query2 = embeddings.embed_query("My package was lost")
query3 = embeddings.embed_query("I want to cancel my order")

print("Package not delivered & package lost:", cosine_similarity(query1, query2))
print("Package not delivered & order cancellation:", cosine_similarity(query1, query3))
print("Package lost & order cancellation:", cosine_similarity(query2, query3))

In [ ]:
# Experiment with the similarity score
sentence1 = "### TODO ###"
sentence2 = "### TODO ###"
cosine_similarity(embeddings.embed_query(sentence1), embeddings.embed_query(sentence2))

#### (Optional) Visualize the embeddings
Even though embeddings are multidimensional vectors it is possible to visualize them on a 2d plane thanks to dimensionality reduction techniques. Below I used PCA from sklearn library to reduce the vectors to 2d space and plot them using matplotlib.

In [ ]:
%pip install scikit-learn matplotlib -qqq

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Collection of texts to compare embeddings. 
# Add your texts to the list.
documents = [
    "My package was not delivered",
    "My package was lost",
    "I want to cancel my order",
    "Today is a very beautiful day",
    "The weather is great"
]
# Compute embeddings using the embedding model. 
vectors = embeddings.embed_documents(documents)
# Dimensionality reduction, allows to get from multidimentional space to 2d.
reduced = PCA(n_components=2).fit_transform(vectors)
# Visualize the results in 2d.
scatter = plt.scatter(reduced[:,0], reduced[:,1], c=list(range(len(documents))))
plt.legend(handles=scatter.legend_elements()[0], labels=documents)
plt.show()